# How does the self-esteem of people in a country change in realtion to cultural attributes such as power distance or collectivist mindsets?

<h2 align="center"> A brief introduction </h2>

The goal of the project is to carry out an end-to-end analysis of a dataset with the focus on statistical analysis and on the visualisation and presentation of results.  

This project will analyse relationships between the self-esteem of a sample of people of a country (extrapolated form the sample offered in the dataset) and the dimensions of national culture power distance and individualism through statistical test and methods.

---
_Finding such relationships can be interesting for managers and team leaders to better understand culutral diversity in their teams and, through this, accomodate different prespectives into the values and organisational culture of a team._

_**From reading of previous studies on the topic, the asusmption was, that countries with a lower power distance and higher individualistic mindset would also have a higher average self-esteem score.**_

---
The main hypothesis was based on the data retrieved from a questionnaire for the self-evaluation of an individual’s self-esteem, developed by Rosenberg in the 60s. The dataset used collected 47,974 answers in total and includes not only the overall score for each participant, but the age and nationality of each.

As a secondary dataset, Hofstede’s cultural compass report was used. This report analyses 6 dimensions of national culture of different countries; power distance, individualism, masculinity, uncertainty avoidance, long-term orientation and indulgence. From here, I selected power distance and individualism as the most relevant dimensions for this project. 


>Both files where extracted from online sources, the Rosenberg test results are form [openpsychometrics](https://openpsychometrics.org/), an open source database.
The report on cultural dimensions was extracted manually from [Hofstedes](https://www.hofstede-insights.com/) webpage. 
Both where in form of a .csv and where edited directly usign python, python librares and sql.

----
##### Limitations of dataset


The main limitation of the sample is in the participant pool. Given that the source of data are results form an online test, whcih requires internet and spare time, it is likely is that the participant  who took the personality test may  consist  of  individuals  who  have  average or above-average  socioeconomic  status,  given  that  all  participants  had access to the Internet and either (a) spoke English, or (b) were able to translate  the  website  into  their  native  language.

The second limitation is the fact that it is a self evaluation of each individual, meaning that some answers may be false or inaccurate.

Lastly, I realised that, when checking for the parameters of the dsitirbutions, I could not find a common distirbution between my RV. This lead to me having to find a non-parametric test for my hypotheiss testing. I ended up using the Kruksal test (an extention of the ANOVA test) for this.

>*However, in theory to get better results, I should have considered my variation and stdev when calculating the mean for each attribute too. The lack of me doing so, might be what lead to my means being so simlar. As we are considering a large dataset with a large variation of answers, the mean ended up being less representatinve as outliers over and under the average cancelled eachouther out. I am planning that once we learn more about regresions, dummies and interpolations I can build on my project to better my results in this area too*



On the other hand, looking at the Hostede report, we have to note that the report looks to analyse a whole countries culutre based off a series of questions, we do not have the answers to given questions, only the conlusions the institution made out of the initial datase. We have to trust the legitimacy and skill of the methods used by Hofstede to state that our analysis is valid. 

----
<h2 align="center"> Data wrangling and DB strucutre</h2>

The data extracted was allready in pretty good shape. All data was in form of easily downloadable files and could easily but uploaded into the jupyter notebook for further work. 

##### The Rosenberg report

As stated above, the dataset was quite clean. 
Following steps where taken to adapt the data to the project's needs:
 1. Addition of all valoration of questions (1 to 4 scale) - to get a scale from 10 to 40
 2. Removing individual question valorations and leaving only the sum
     - Only the sum will be used for the final analysis 
 3. Changing the gender to M/F/O for better understandign of data
 4. Removing any scores under 10 (if score < 10, it means some of the questions where not answered, as only a few rows where affected, they where removed.
 5. All individuals who where not between the ages of 10 to 85 where also removed. 
 6. To ensure the validity of the sample size, for countries which  had less than 18 answers (the median value), all answers where removed. 
    - The median value was used as the data was strongly skewed (50% of responses came from the US) and therefore the median is not representative
 5. Change the name of the country by the Country_id to have a common foreign key in the DB

##### Hofstede report

Here too, the data was very clean, apart from some NaNs, where I dropped the whole rows (as NaNs tended to be consistent for a row in all variables), and some column renaming, not much was done. 
The country name was changed to country_id, and collective gruppations fo countries into regions (Yugoslavia, East Africa, West Africa, Arabic Countries) where removed as they do not appear in the country list. 

----
### The DataBase

All cleaned tables where then uploaded onto the google cloud SQL server.
The layout of the Database goes as follows:
![DB Layout plan](./01-DB_structure_diagramm.PNG)
As all tables have country as a variable, **Country was used as an foreign key** to work with the tables. This required some additional work as the nomination on nationas differed form study to study. Finally I decided on using the [ISO 3166 naming skeme](https://en.wikipedia.org/wiki/List_of_ISO_3166_country_codes), which was also the skeme used for the Rosenberg survey. The ISO naming table could also be downloaded as a .csv, making life much easier.

From there, I made a dictionary and simply replaced the country names with the index of the ISO skeme DataFrame. 



<div class="alert alert-block alert-info">
<b>Credentials for DB:</b> 

##### To access the databse see the details below:
Note that this database has an IP restriction, and can therefore only be viewed in the IRONHACK campus. If the DB needs to be accessed from a differnt IP adress, please contact me. 

>**Connection information:** <br>
>**user name**: ironhack <br>
>**passowrd**: Ironhack1 <br>
>**host name**: 35.240.116.117 <br>
</div>


---
<h2 align="center"> The analysis</h2> 

### Initial hypothesis

>_**H0:** Null hypothesis assumes that the samples (groups) are from identical populations <br>
<br>
<br>
**H1:** Alternative hypothesis assumes that at least one of the samples (groups) comes from a different population than the others_



The initial step to solve this quesiton, is through the definition of the distribution of the data. 

This was done first, thorugh the representaiton of the data in form of a histogram, and then through a simple normal distribution test. 

The test were carried out with the H0 that the df follows a normal distribution.

**i.e. The value exepcted of the thest was a p > 0.05 for our H0 to be accepted with a 95% confidence.**

From our results, although when plotting the graphs we see that the distribution looks pretty much normal, the test prove otherwise. Neither the results obtained from a sample of randomly chosen individual countries, nor the distribution of the average score when grouping by country, match this requirement. 

*See the code below for reference*

In [ ]:
# check for normal distribution
for i in lst:
    df = df_rosenberg.loc[df_rosenberg["country"] == i]["overall_score"]
    print (stats.normaltest(df))

The same was done to see if the distirbution follows a chi squared or lognormal distribution, however, **none of the H0s could be rejected**. 

This initially left me at a loss, especially after carrying out an anderson test (provides insights into what distribution the data may follow), and the test came out with a completely different result for each of the randomly selected random varibales. 

After some further research, I realised that I was working with non parametric samples, meaning that I had to rely on non-parametric hypothesis testing for my significance tests (see [link](https://www.statisticshowto.datasciencecentral.com/parametric-and-non-parametric-data/)) for more info on this.

At this point I also came to the conclusion that I should probably be using a more complex regression has to be carried out (probably some sort of non parametric hypothesis testing) to see any results. 

*See the code below for reference*

In [ ]:
# check for chi squared distribution
for i in lst:
    df = df_rosenberg.loc[df_rosenberg["country"] == i]["overall_score"]
    print (stats.normaltest(df).pvalue)

# check for lognormal distribution    
import math
for i in lst:
    df = df_rosenberg.loc[df_rosenberg["country"] == i]["overall_score"]
    print (stats.normaltest(df.apply(lambda x:np.exp(x))).pvalue)  
    
# check for distribution type:    
for i in lst:
    df = df_rosenberg.loc[df_rosenberg["country"] == i]["overall_score"]
    print (stats.anderson(df)[0])

"""The following answers where returned: 
19.730, 165.720, 5.70092, 54.714, 165.720, 2.745, 30.029"""

I finally chose to carry out a kruskal test, as it is the only non-aprametric test readily available in the scipy library. Althouhg I am aware that, if at all possible, you should us parametric tests, as they tend to be more accurate, nonparametric tests can perform well with non-normal continuous data if you have a sufficiently large sample size, which is the case in my dataset. 

----

### 1. Grouping individuals by country (sample mean) - is there a relation to individualism?

>_**H0:** Null hypothesis assumes that the samples (groups) are from identical populations <br>
>>I.e. there is no significant difference in self esteem between countries with an individualistic mindset vs countries with a collective mindset (i.e. the samples are form indentical populations) <br>

>**H1:** Alternative hypothesis assumes that at least one of the samples (groups) comes from a different population than the others <br>
>>I.e. there is a significant difference in the self esteem score, this **could possibly** mean that the individualistic or collective mindset of society can have an effect on self esteem (TEST IS NOT CAUSAL PROOF)_

The analysis was started by selecting the Random Variables which would be used to test the hypothesis.

After ordering the variables into high individualism and low individualism, two countries where chosen, their distirbutions mapped and compared through a Kruskal test. 
The test was repeated three times on three different paris of coutnries to ensure that it was not just mere coincidence that I obtain the results I got.

The code used is presented below, for more detail view the notebook 03-Analysis. 

In [ ]:
# VENEZUEL (low) vs. USA (high)
sample_1 = df_rosenberg.loc[df_rosenberg["country_id"] == 239]["overall_score"]
sample_2 = df_rosenberg.loc[df_rosenberg["country_id"] == 234]["overall_score"]

# asuming alpha = 0.05
print(stats.kruskal(sample_1, sample_2))

"""KruskalResult(statistic=0.23282120705540457, pvalue=0.6294397907001184)
p value >> a, H0 CANNOT BE REJECTED"""

For a large magority of the tesitng, the p value was greater than the alpha value given. This means that the H0 CANNOT be rejected, there is no siginificant difference in the populations of both countries when looking at the self esteem score. 

>**Overall, H0 cannot be rejected, the samples come from an identical polulation (with a 95% confidence)**

----
### 2. Grouping individuals by country (sample mean) - is there a relation to power-distance?

>_**H0:** Null hypothesis assumes that the samples (groups) are from identical populations <br>
>>I.e. there is no significant difference in self esteem between countries with a high power distance vs countries with a lowe power distance (i.e. the samples are form indentical populations) <br>
<br>

>**H1:** Alternative hypothesis assumes that at least one of the samples (groups) comes from a different population than the others <br>
>>I.e. there is a significant difference in the self esteem score, this **could possibly** mean that the cultural attribute can have an effect on self esteem (TEST IS NOT CAUSAL PROOF)_

Here, the same approach was used as above. The coutntries were grouped into high and low powerdistance and then compared to eachother. Three countries from each group were selected and tested for a significant difference in populaiton.

The code used is presented below, for more detail view the notebook 03-Analysis. 

In [ ]:
#Austria, Slovakia 

sample_1 = df_rosenberg.loc[df_rosenberg["country_id"] == 12]["overall_score"]
sample_2 = df_rosenberg.loc[df_rosenberg["country_id"] == 203]["overall_score"]

# asuming alpha = 0.05
print(stats.kruskal(sample_1, sample_2))

"""KruskalResult(statistic=2.281086923087371, pvalue=0.1309600666415828)
p value > a, H0 CANNOT BE REJECTED"""

>**H0 cannot be rejected, the samples come from an identical polulation (with a 95% confidence)**

----
### 3. Finding correlations and realtionships between datasets

After some consideration, I decided to not include a correlation analysis in the final paper, although it appears in the notebook, as we are handling RV with different distributions. To present the data to the audiance, I have plotted the data in a scatter diagram to show any sort of approximation to a trend, the results of this can be seen in the section below *(see Insights, Correlations and Trends)*.

>I will look into findign relationships between this manner of datasets in the future.<br>
A possible way to analyse the realtionship might be through a gamma regression, which tells us the strength of association between points.The goal of the test is to tell us whereadditional values would rank. As regressions where not covered in this module of the course, and tend to involve machine learning aspects of coding, I have left this part out. <br>
>>For more inofrmation on the Gamma Coefficient see here for the statistical explanation: [gamma coefficient: Statistics How To](https://www.statisticshowto.datasciencecentral.com/gamma-coefficient-goodman-kruskal/) and here for the programming explanation: [gamma coding](https://tsmatz.wordpress.com/2017/08/30/glm-regression-logistic-poisson-gaussian-gamma-tutorial-with-r/)


----
### 4. Digging deeper into the data

Being slightly dissapointed that I got no significant results, I then decided to analyse some of the other cathegories I had in my dataset. 

The fist was an analysis by gender, wanting to define weather gender changes the mean sample score, and the second was an analysis by age, wanting to find out if and individuals self-esteem changed over ones lifespan. 


>When looking at the aga analysis, it is important to mention again, similarily to the correaltion analysis, that the results would have been more representative of the data if a regression that takes into account the variance would have been carried out.  



##### Gender

For the analysis on gender, I decided to simply graph a frequency distribution and look at the modal score. 

The data was grouped using a simple crosstab function **pd.crosstab**, creating a count per self-esteem score by gender. From there, I could then plot a graph with the frequecny (y-axis) of each score (x-axis). The graph and conclusions for this can be seen below in the Insights section. 

Out of pure interest, I then wanted to find out if the changes in distribution where significant enough. However, a quick kruskal test showed that eventually no, there is no significant change in population:

In [ ]:
# quick check for H0: same population vs. H1: different population
sample_1 = F["F"]
sample_2 = M["M"]

# asuming alpha = 0.05
print(stats.kruskal(sample_1, sample_2))

"""KruskalResult(statistic=0.8382657031503321, pvalue=0.359893252079605)
p value > a, H0 CANNOT BE REJECTED
"""

##### Age

The analysis of age presented itself as a bit more of a challenge than I thought it would. 
I initially simply grouped by age, calculated the mean and created a scatter diagram with a lowess trendline. This allready gave me great insights into the data (see below to find the Insights). 
However, jsut as above, when calculating the realationship in the data, I believe I would have been able to obtain more representative reults using regressions and machine learning.  

____
<h2 align="center"> Insights</h2> 

For our insights, the first representation done was a geographic heat map to visually recognise the regions with higher and lower self-esteem.
This was done using the plotly express library and changing the country codes to ADM0_A3 codes, as that is the code used by the library. 
the results of the interactive map can be seen in the analysis notebook. The screenshot can be seen below:

![map](./graphs/worldmap.png)


Through this analysis, we can allready start seeing the initial hints that our assumptions might be wrong. Here we can see the first surprises, countries with a high power distance and a collective mindset, such as china, score relatively high on the scale, whilst countries like the US, indivisualistic and mid-range in power distribution, score lowly.

### 1. Hypothesis testing
The main insight in our hypothesis testing is that the H0 cannot be rejected, neither when looking at power distribution, nor when looking at the individualism of societies. <br>
This is representative of the map shown above. 

This basically means that, when considering a 95% certainty interval, there is not enough significant difference to reject the initial hypothesis that assumes that the samples are from identical population.

### 2. Correlations and trends
Even though I chose not focus on a more complex realtional analysis, I did present two scatter diagrams for the understanding of what a dataset with little correaltion looks like. The two graphs, as well as the corraltion table can be seen below:

![scatter diagram 1](./graphs/scatter_ind.svg)


![scatter diagram 2](./graphs/scatter_pd.svg)


![corr](./graphs/correlation_no_map.png)

### 3. Bonus: looking at other variables (gender and age)

Below the distributions of self-esteem can be seen per sex. 

It is very surprising that the modal score is higher in the female proportion of respondants than in the male. Typically one would assume the opposite to be true. 
![fem](./graphs/female.png) ![mal](./graphs/male.png)

Lastly, the progress of self-esteem over age was plotted and a line of best fit added. Again, please bear in mind that this would be more representative if a regression would have been carried out. 

I believe that it is due to this, that the surprising result comes out that one average, the ages where one is the msot confident is in the early 20s, and it drops from there. It is however interesting to notice, that the scatterdiagram shows a very consistent parttern, from the ages of 10 to around 60, the mean answer for all ages seems to follow a consisten path.


![age](./graphs/age.png)

----
<h2 align="center"> Afterthoughts</h2> 

There is a lot to expand in this project, especially when extending the research to look for causality instead of only the realtionship between variables. 

It would also be intersting to see the relationship between self esteem and pressure on Teens and YA, focusing more on the psycological aspects than on cultural attributes. 


For more information on the topic, check out the rest of the work for the project:

[Presentation link](https://docs.google.com/presentation/d/1X4Y6I8M2PAm_L0lgAkckxEc9Z16ysNbOIY9auYnJv4g/edit?usp=sharing)

[My notebooks and codes](https://github.com/FHnt97/Project-Week-5-Your-Own-Project)

If you are interested in reading more about the topic, here is some interesting studies and reports you might enjoy:

[The culture orientation scale](https://fetzer.org/sites/default/files/images/stories/pdf/selfmeasures/CollectiveOrientation.pdf)

[Age, gender, and self-esteem](https://psycnet.apa.org/fulltext/2017-36143-001.pdf)


[Individualistic cultures and behaviour](https://psycnet.apa.org/fulltext/2017-36143-001.pdf)
